cf. 
- https://stackoverflow.com/questions/56888333/how-can-i-parse-a-wikipedia-xml-dump-with-python
- https://github.com/mediawiki-utilities/python-mwxml
- https://github.com/mediawiki-utilities/python-mwxml/blob/master/ipython/labs_example.ipynb

Wikipedia dump version in use:

https://dumps.wikimedia.org/jawiki/20250401/

In [1]:
import mwxml
from tqdm import tqdm

In [2]:
dump = mwxml.Dump.from_file(
    open('data/wiki_ja.xml', 'r')
)

In [ ]:
page_text_list = []
last_visit_page_idx = 1
for page_idx, page in tqdm(enumerate(dump, start=1)):
    for revision in page:
        # Because what we have downloaded is the dump of each page, WITHOUT revision history.
        # So there should be only one "revision" objectg in the "page" object.
        # cf. the raw XML for the file structure.
        # We dump each article separately for the conveinence of future processing.
        if revision.text is not None:
            page_text_list.append(page.title)
            page_text_list.append(revision.text)
        if ( (page_idx % 10000) == 0 ):
            # Notice that, the dump variable is an iterator.
            # Therefore, we dropped last 10,000 articles in the dump data.
            # The total number of articles is 2,979,559. It's fine.
            with open('data/text/article_' + str(last_visit_page_idx) + '-' + str(page_idx) + '.txt', 'w', encoding='utf-8') as f_handle:
                for text in page_text_list:
                    f_handle.write(text)
                    f_handle.write('\n')
                last_visit_page_idx = page_idx
                page_text_list = []

2979559it [10:43, 4629.59it/s]
